# Chat with Your Data


1. Library installs

In [4]:
! pip install 
! pip install openai
! pip install pypdf

ERROR: You must give at least one requirement to install (see "pip help install")


2. Setup openAI API

In [ ]:
import os 
import openai 

# get api key from system environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")
